### DENSENET 211
- Training Dataset: RandAugment, N=2, M=20
- Sagemaker Notebook must be of type, conda_pytorch_p36

In [ ]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

#### Install Requirements

In [9]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
# Need to add this to requirements.txt
!pip install tensorboard

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/densenet.yaml \
    model.densenet.depth 100 \
    model.densenet.growth_rate 12 \
    train.batch_size 64 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_RA_2_20 \
    train.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00 \
    scheduler.epochs 400

[2020-07-01 01:47:45] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_2_20
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: densenet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottlenec

In [11]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00_resume400_50 \
    scheduler.epochs 50


[2020-07-02 03:45:48] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: densenet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_

In [12]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00_resume400_50/test_results_0050_cifar10

[2020-07-02 12:22:41] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  8.17it/s]
[2020-07-02 12:23:01] __main__ INFO: Elapsed 19.22
[2020-07-02 12:23:01] __main__ INFO: Loss 0.3482 Accuracy 0.9123


In [13]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/test_results_0400_cifar10

[2020-07-02 12:24:06] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  8.21it/s]
[2020-07-02 12:24:26] __main__ INFO: Elapsed 19.13
[2020-07-02 12:24:26] __main__ INFO: Loss 1.0119 Accuracy 0.8154


In [14]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00_resume400_50/test_results_0050_cifar101

[2020-07-02 12:25:26] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.67it/s]
[2020-07-02 12:25:31] __main__ INFO: Elapsed 4.17
[2020-07-02 12:25:31] __main__ INFO: Loss 0.7125 Accuracy 0.8315


In [15]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Write the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/densenet.yaml \
    model.densenet.depth 100 \
    model.densenet.growth_rate 12 \
    test.batch_size 64 \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/checkpoint_00400.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/test_results_0400_cifar101

[2020-07-02 12:28:28] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.64it/s]
[2020-07-02 12:28:33] __main__ INFO: Elapsed 4.19
[2020-07-02 12:28:33] __main__ INFO: Loss 1.8176 Accuracy 0.6815


In [16]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/checkpoint_00300.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/test_results_0300_cifar10

[2020-07-02 12:29:35] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/checkpoint_00300.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  8.19it/s]
[2020-07-02 12:29:55] __main__ INFO: Elapsed 19.18
[2020-07-02 12:29:55] __main__ INFO: Loss 0.9771 Accuracy 0.8119


In [17]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/checkpoint_00300.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/test_results_0300_cifar101

[2020-07-02 12:30:19] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/checkpoint_00300.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.64it/s]
[2020-07-02 12:30:23] __main__ INFO: Elapsed 4.19
[2020-07-02 12:30:23] __main__ INFO: Loss 1.7810 Accuracy 0.6745


In [ ]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['densenet_BC_100_12', 'densenet_BC_100_12', 'densenet_BC_100_12', 'densenet_BC_100_12'],
           'Testset': ['cifar10', 'cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 200, 300, 300],
           'Loss': [0.3681, 0.2279, 0.2223, 0.5098],
           'Accuracy': [0.8875, 0.9456, 0.9484, 0.8830],
           'Original_Accuracy': [95.5, 95.5, 95.5, 87.6],
           'Original_CI': [(95.1, 95.9), (95.1, 95.9), (95.1, 95.9), (86.1, 89.0)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])


df.to_csv('/home/ec2-user/SageMaker/experiments/densenet_BC_100_12/exp00/results.csv')
df.head()

In [21]:
import pandas as pd
a = pd.Series(['densenet_BC_100_12_ra_2_20_c10val', 400, 'cifar10', 1.0119, 0.8154])
b = pd.Series(['densenet_BC_100_12_ra_2_20_c10val', 300, 'cifar10', 0.9771, 0.8119])
c = pd.Series(['densenet_BC_100_12_ra_2_20_c10val', 400, 'cifar10.1', 1.8176, 0.6815])
d = pd.Series(['densenet_BC_100_12_ra_2_20_c10val', 300, 'cifar10.1', 1.7810, 0.6745])

e = pd.Series(['densenet_BC_100_12_ra_2_20_refined400_c10val', 50, 'cifar10.1', 0.7125, 0.8315])
f = pd.Series(['densenet_BC_100_12_ra_2_20_refined400_c10val', 50, 'cifar10', 0.3482, 0.9123])
# g = pd.Series(['resnet_basic_32_ra_2_5_refined300', 150, 'cifar10', 0.4499, 0.8795])
# h = pd.Series(['resnet_basic_32_ra_2_5_refined300', 150, 'cifar10.1', 0.8206, 0.7710])
               
df_results = pd.concat([a,b,c,d,e,f], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 95.5 if row[2] == 'cifar10' else 87.6), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (95.1, 95.9) if row[2] == 'cifar10' else (86.1, 89.0)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,densenet_BC_100_12_ra_2_20_c10val,400,cifar10,1.0119,0.8154,95.5,"(95.1, 95.9)"
1,densenet_BC_100_12_ra_2_20_c10val,300,cifar10,0.9771,0.8119,95.5,"(95.1, 95.9)"
2,densenet_BC_100_12_ra_2_20_c10val,400,cifar10.1,1.8176,0.6815,87.6,"(86.1, 89.0)"
3,densenet_BC_100_12_ra_2_20_c10val,300,cifar10.1,1.781,0.6745,87.6,"(86.1, 89.0)"
4,densenet_BC_100_12_ra_2_20_refined400_c10val,50,cifar10.1,0.7125,0.8315,87.6,"(86.1, 89.0)"
5,densenet_BC_100_12_ra_2_20_refined400_c10val,50,cifar10,0.3482,0.9123,95.5,"(95.1, 95.9)"


In [ ]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val/exp00/test_results_0400/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

In [20]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-june29'
prefix = 'sagemaker/results/original-models/densenet_BC_100_12_ra_2_20_c10val'
path = '/home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_20_c10val'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)